<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AAI_MEDICAL_DEEPSEEK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import json
from openai import OpenAI
from google.colab import userdata

# --- API Configuration ---
api_key = userdata.get('DEEPSEEK_API_KEY')
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com/v1")

# --- Tool Definitions ---
# These are the functions that the Executor Agent can call.
def get_patient_ehr(patient_id: str) -> dict:
    """Simulates retrieving a patient's Electronic Health Record."""
    ehr_data = {
        "67890": {
            "name": "John Smith",
            "age": 62,
            "gender": "Male",
            "past_history": "Coronary Artery Disease, Asthma",
            "allergies": "None",
            "current_medications": ["Aspirin", "Albuterol inhaler"],
        }
    }
    return ehr_data.get(patient_id, {"error": "Patient not found"})

def order_ecg(patient_id: str, reason: str) -> str:
    """Simulates ordering an ECG for a patient."""
    return f"ECG order for patient {patient_id} placed. Reason: {reason}. Results will be available shortly."

def get_ecg_results(patient_id: str) -> dict:
    """Simulates getting ECG results for a patient."""
    return {"patient_id": patient_id, "findings": "Sinus tachycardia with ST depression in leads V3-V4."}

def order_cardiac_enzymes(patient_id: str, reason: str) -> str:
    """Simulates ordering cardiac enzymes."""
    return f"Cardiac enzyme panel ordered for patient {patient_id}. Reason: {reason}. Results will be available shortly."

def recommend_cardiology_consult(patient_id: str, reason: str) -> str:
    """Simulates recommending a cardiology consult."""
    return f"Cardiology consult for patient {patient_id} recommended. Reason: {reason}."

# Corrected tool definitions with proper JSON schema
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_patient_ehr",
            "description": "Retrieves detailed Electronic Health Record for a patient.",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {"type": "string"}
                }, "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "order_ecg",
            "description": "Places an order for an ECG.",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {"type": "string"},
                    "reason": {"type": "string"}
                }, "required": ["patient_id", "reason"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_ecg_results",
            "description": "Retrieves the ECG results for a patient.",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {"type": "string"}
                }, "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "order_cardiac_enzymes",
            "description": "Places an order for cardiac enzymes.",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {"type": "string"},
                    "reason": {"type": "string"}
                }, "required": ["patient_id", "reason"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "recommend_cardiology_consult",
            "description": "Recommends a cardiology consult for a patient.",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {"type": "string"},
                    "reason": {"type": "string"}
                }, "required": ["patient_id", "reason"]
            }
        }
    }
]

# --- Main Orchestration Logic ---
def run_agentic_flow_corrected(user_prompt: str, patient_id: str):
    """
    Main function to orchestrate the multi-agent workflow correctly.
    """
    try:
        # Step 1: Initialize the conversation with the user prompt.
        messages = [
            {"role": "system", "content": "You are a clinical decision support agent. You must use your tools to gather all necessary information before providing a final, synthesized recommendation to the user."},
            {"role": "user", "content": f"{user_prompt}. Patient ID: {patient_id}"}
        ]

        # Step 2: The Orchestrator manages the conversation turns until a final answer is given.
        for _ in range(5):  # Limit turns to prevent infinite loops
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=tools,
                tool_choice="auto"
            )

            response_message = response.choices[0].message
            messages.append(response_message)

            if not response_message.tool_calls:
                # The model did not request a tool call, so it's the final answer.
                return response_message.content

            print("\nModel has decided to call tools...")

            # Step 3: Execute the planned tool calls.
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)
                tool_call_id = tool_call.id

                print(f"Executing tool: {function_name} with arguments: {arguments}...")

                # Call the local function based on the model's request.
                if function_name == "get_patient_ehr":
                    tool_response_content = get_patient_ehr(**arguments)
                elif function_name == "order_ecg":
                    tool_response_content = order_ecg(**arguments)
                elif function_name == "get_ecg_results":
                    tool_response_content = get_ecg_results(**arguments)
                elif function_name == "order_cardiac_enzymes":
                    tool_response_content = order_cardiac_enzymes(**arguments)
                elif function_name == "recommend_cardiology_consult":
                    tool_response_content = recommend_cardiology_consult(**arguments)
                else:
                    tool_response_content = {"error": "Unknown tool."}

                # Step 4: Add the tool's result to the conversation history.
                messages.append(
                    {
                        "tool_call_id": tool_call_id,
                        "role": "tool",
                        "name": function_name,
                        "content": json.dumps(tool_response_content)
                    }
                )
        return "Max conversation turns reached without a final answer."

    except Exception as e:
        return f"An error occurred: {e}"

# --- Execution ---
if __name__ == "__main__":
    patient_query = "A 62-year-old male presents with chest pain and shortness of breath. He is concerned about his heart health. What should the next steps be?"
    patient_id = "67890"

    print("--- Agentic Clinical Decision Support System ---")
    print(f"User Query: {patient_query}\n")

    result = run_agentic_flow_corrected(user_prompt=patient_query, patient_id=patient_id)
    print("\n--- Final Agent Recommendation ---")
    print(result)

--- Agentic Clinical Decision Support System ---
User Query: A 62-year-old male presents with chest pain and shortness of breath. He is concerned about his heart health. What should the next steps be?


Model has decided to call tools...
Executing tool: get_patient_ehr with arguments: {'patient_id': '67890'}...

Model has decided to call tools...
Executing tool: order_ecg with arguments: {'patient_id': '67890', 'reason': 'Chest pain and shortness of breath in a 62-year-old male with known coronary artery disease'}...

Model has decided to call tools...
Executing tool: order_cardiac_enzymes with arguments: {'patient_id': '67890', 'reason': 'Evaluation of chest pain and shortness of breath in patient with coronary artery disease'}...

Model has decided to call tools...
Executing tool: get_ecg_results with arguments: {'patient_id': '67890'}...

--- Final Agent Recommendation ---
Based on my evaluation of this 62-year-old male patient (John Smith), here is my synthesized recommendation:

#